In [ ]:
import os
import sys
import glob
import shutil
from copy import deepcopy
from ipywidgets import interact
import matplotlib.pyplot as plt
import numpy as np

sys.path.append('../..')
from lung_cancer import CTImagesBatch
from lung_cancer.dataset import FilesIndex, DatasetIndex
print('OK')

### Global index

In [ ]:
DIR_TEST = '/notebooks/data/MRT/nci/*'
DIR_DUMP = '/notebooks/Koryagin/Batch_dev/dir_for_dumps/'

In [ ]:
if os.path.exists(DIR_DUMP):
    shutil.rmtree(DIR_DUMP)

In [ ]:
ind = FilesIndex(path=DIR_TEST, dirs=True)

### Create batch by subindexing

In [ ]:
batch = CTImagesBatch(ind.create_subset(ind.index[[2,5,7]]))

In [ ]:
isinstance(batch.index, DatasetIndex)

In [ ]:
isinstance(batch.indices, DatasetIndex)

In [ ]:
batch.indices

### Test actions

* check load

In [ ]:
batch.load(fmt='dicom')

In [ ]:
batch_copy = deepcopy(batch)

* check attrs

In [ ]:
batch.shape

In [ ]:
batch._bounds

In [ ]:
batch.upper_bounds

* check resize, segment, dump

In [ ]:
# note that args-order is z y x, just like _data is stores in batch
batch = batch.resize(shape=(128, 256, 256)).segment()

In [ ]:
batch._bounds

* check dump

In [ ]:
batch.dump(dst=DIR_DUMP)

### Build batch from dump and compare

In [ ]:
ind_dumped = FilesIndex(path=DIR_DUMP + '/*', dirs=True)

In [ ]:
batch_from_dump = CTImagesBatch(ind_dumped)

In [ ]:
batch_from_dump.load(fmt='blosc')

In [ ]:
def plot_arr_slices(height, *arrays, clim=(-1200, 300)):
    fig, axes = plt.subplots(1, len(arrays), figsize=(14, len(arrays)*8))
    
    for arr, i in zip(arrays, range(len(arrays))):
        depth = arr.shape[0]
        n_slice = int(depth * height)
        
        kwargs = dict()
        if np.max(arr) - np.min(arr) > 2.0:
            kwargs.update(clim=clim)
        else:
            kwargs.update(clim=(0, 1))
    
        axes[i].imshow(arr[n_slice], cmap=plt.cm.gray, **kwargs)
    plt.show()

In [ ]:
n_pat = 0
ind_npat = batch.indices[n_pat]

In [ ]:
interact(lambda height: plot_arr_slices(height, 
                                        batch.get(ind_npat, 'images'),
                                        batch_copy.get(ind_npat, 'images'),
                                        batch_from_dump.get(ind_npat, 'images')),
         height=(0.01, 0.99, 0.01))

### XIP

In [ ]:
xip = batch.make_xip()

In [ ]:
interact(lambda height: plot_arr_slices(height, 
                                        batch.get(ind_npat, 'images'),
                                        xip.get(ind_npat, 'images')),
         height=(0.01, 0.99, 0.01))

### Splitting/assembling into/from patches

* split each scan into patches of shape=(64, 128, 128) with stride=(32, 64, 64) 

In [ ]:
patches = batch.get_patches((64, 128, 128), (32, 64, 64))

In [ ]:
patches.shape

* assemble patches into scan of shape=(128, 256, 256) using stride=(32, 64, 64) and load the result into 'images_2'-attr of batch

In [ ]:
batch.load_from_patches(patches, (32, 64, 64), (128, 256, 256), 'images_2')

* check that patches are assembled into the same scan

In [ ]:
np.sum(np.abs(batch.images_2 - batch.images))